In [1]:
# PyAlex and Data Creation imports
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import pyalex, os, csv
pyalex.config.email = "b.cliff@gwmail.gwu.edu"
from itertools import chain

In [2]:
# Data Validation imports
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List

#### Configuring Output Files

In [3]:
WORKS_FILE = './data/works.csv'
AUTHORS_FILE = './data/authors.csv'
TOPICS_FILE = './data/topics.csv'

In [4]:
# Removing any old/vestige files
for path in [WORKS_FILE, AUTHORS_FILE, TOPICS_FILE]:
    if os.path.exists(path):
        os.remove(path)

In [5]:
works_file = open(WORKS_FILE, 'w', newline='',encoding='utf-8')
authors_file = open(AUTHORS_FILE, 'w', newline='', encoding='utf-8')
topics_file = open(TOPICS_FILE, 'w', newline='', encoding='utf-8')

In [6]:
# Creating the headers
works_writer = csv.writer(works_file)
authors_writer = csv.writer(authors_file)
topics_writer = csv.writer(topics_file)

In [ ]:
# Writing the headers to the csv files
works_writer.writerow(['id', 'doi', 'title', 'publication_date', 'type', 'related_works', 'corresponding_author_ids'])

In [7]:
authors_writer.writerow(['id', 'name', 'works_count', 'cited_by_count', 'affiliations', 'topics', 'counts_by_year'])
# topics_writer.writerow([])

71

#### Creating Pydantic Models for Validation

In [8]:
class WorksModel(BaseModel):
    id: str
    doi: Optional[str] = None
    title: str
    publication_date: str
    type: Optional[str] = None
    related_works: List[str] = None
    corresponding_author_ids: List[str] = None

In [9]:
class AuthorsModel(BaseModel):
    id: str
    display_name: str
    works_count: int
    cited_by_count: int
    affiliations: List[dict] = None
    topics: List[dict] = None
    counts_by_year: List[dict]

In [10]:
class TopicsModel(BaseModel):
    id: str
    display_name: str
    description: str
    keywords: List[str] = None
    subfield_name: Optional[str] = None
    field_name: Optional[str] = None
    domain_name: Optional[str] = None
    works_count: int
    cited_by_count: int
    updated_date: str
    created_date: str

### Data retrieval from OpenAlex

In [11]:
oa_replace = 'https://openalex.org/'

In [12]:
works_pager = Works().select(['id', 'doi', 'title',
'publication_date', 'type', 'related_works', 'corresponding_author_ids']).paginate(per_page=200)

In [13]:
authors_pager = Authors().select(['id', 'orcid', 'display_name',
                                  'works_count', 'cited_by_count',
                                  'affiliations', 'topics', 'counts_by_year']).paginate(per_page=200)

In [14]:
topics_pager = Topics().select(['id', 'display_name', 'description', 'keywords',
                                'subfield', 'field', 'domain', 'works_count',
                                'cited_by_count', 'updated_date', 'created_date']).paginate(per_page=200)
# not including siblings for now - complicated

#### Data Cleaning

In [15]:
for page in works_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            raw['related_works'] = [rel.replace(oa_replace, '') for rel in raw['related_works']]
            raw['corresponding_author_ids'] = [rel.replace(oa_replace, '') for rel in raw['corresponding_author_ids']]
            w = WorksModel(**raw)
            works_writer.writerow(
                [w.id,
                w.doi,
                w.title,
                w.publication_date,
                w.type,
                w.related_works,
                w.corresponding_author_ids]
            )
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
works_file.close()


In [16]:
for page in authors_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            a = AuthorsModel(**raw)
            authors_writer.writerow([
                a.id,
                a.display_name,
                a.works_count,
                a.cited_by_count,
                a.affiliations,
                a.topics,
                a.counts_by_year
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
authors_file.close()

In [18]:
topics_file.close()

In [30]:
rand_A = Authors().random()

#### Including Filter and Select

In [ ]:
# Determining the keys for each Works() object
Works()['W2889162861'].keys()

In [ ]:
Works()['W2889162861']['abstract_inverted_index']

In [ ]:
for work in Works().get():
    if work['authorships'] != []:
        print(work)
        break